In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


In [ ]:
# Load and preprocess the dataset
def load_dataset(data_dir):
    data = []
    labels = []
    categories = ["with_mask", "without_mask"]

    for category in categories:
        path = os.path.join(data_dir, category)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (224, 224))
            image = preprocess_input(image)
            data.append(image)
            labels.append(category)
data = np.array(data, dtype="float32")
    labels = np.array(labels)

    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = np.array(labels)

    return train_test_split(data, labels, test_size=0.2, stratify=labels)


In [ ]:
# Load data
trainX, testX, trainY, testY = load_dataset("dataset")  # dataset/with_mask, dataset/without_mask

# Data augmentation
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# Build model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=baseModel.input, outputs=output)


In [ ]:
# Freeze base model layers
for layer in baseModel.layers:
    layer.trainable = False


In [ ]:
# Compile
model.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
